### Installation des bibliothèques 

pip install numpy  
pip install pandas 
pip install scikit-learn
pip install torch
pip install transformers
pip install datasets

In [36]:
pip install torch torchvision


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.7 MB/s eta 0:00:00a 0:00:01
Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from datasets import Dataset
from tqdm import tqdm
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, Trainer,TrainingArguments, BertTokenizerFast
from sklearn.preprocessing import OneHotEncoder


/home/yop/project_dumas2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

## Data


In [3]:
file1='Data clear/fichier_combiné.csv'
df = pd.read_csv(file1)
print(df.shape,df.head())

(33471, 2)                                                Block  Label
0  ﻿The Project Gutenberg eBook of Contes Françai...      3
1  You may copy it, give it away or re-use it und...      3
2  If you are not located in the United States, y...      3
3  Title: Contes Français Editor: Douglas Labaree...      3
4                                                 D.      3


In [4]:
# Mélanager le dataframe
df_suffle = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df_suffle.shape)
print(df_suffle.head())

(33471, 2)
                                               Block  Label
0  Un cordon de grosses pierres fixait tout autou...      1
1  Vous dirai-je pourtant que j'ai perdu quelques...      2
2  lèvre, _f._ lip; lèvres en fleur, full-blown l...      3
3  Bonaparte (Napoléon), the French Emperor (1769...      3
4  mengheldichten: fyghes noeper;  Bacchus-Cortryck.      2


In [5]:
# Convertir les label du types int en type float 
df_suffle['Label']=df_suffle['Label'].astype(float)

### 1. Encode labels


In [6]:
one=OneHotEncoder(sparse_output=False)
encoded=one.fit_transform(df_suffle[['Label']])
len(encoded)

33471

### 2. Creat new data with the encoded value

In [7]:
head = {'text' : [], 'label' : []}
df_model = pd.DataFrame(head,dtype=object)

In [8]:
for k in tqdm(range (len(df))):
    df_model.loc[k]=[df_suffle['Block'][k],encoded[k]]
df_model.head(10)

  0%|          | 152/33471 [00:00<00:21, 1519.85it/s]

100%|██████████| 33471/33471 [00:35<00:00, 944.03it/s] 


,text,label
0,Un cordon de grosses pierres fixait tout autou...,"[0.0, 1.0, 0.0, 0.0]"
1,Vous dirai-je pourtant que j'ai perdu quelques...,"[0.0, 0.0, 1.0, 0.0]"
2,"lèvre, _f._ lip; lèvres en fleur, full-blown l...","[0.0, 0.0, 0.0, 1.0]"
3,"Bonaparte (Napoléon), the French Emperor (1769...","[0.0, 0.0, 0.0, 1.0]"
4,mengheldichten: fyghes noeper; Bacchus-Cortryck.,"[0.0, 0.0, 1.0, 0.0]"
5,Vous aviez donc des livres?,"[1.0, 0.0, 0.0, 0.0]"
6,Ah çà!,"[0.0, 1.0, 0.0, 0.0]"
7,"faire, _v._ to do, make, form, build, perform,...","[0.0, 0.0, 0.0, 1.0]"
8,Sa présence me rendit ma fermeté.,"[0.0, 0.0, 0.0, 1.0]"
9,"grouiller, _v._ to swarm, crawl.","[0.0, 0.0, 0.0, 1.0]"


In [9]:
print(type(df_model['text'][2]))
print(type(df_model['label'][0]))

<class 'str'>
<class 'numpy.ndarray'>


### 3. Split the data into train and test

In [10]:
#Split data
train_df, test_df = train_test_split(df_model,test_size=0.2)
print(len(train_df),len(test_df))

26776 6695


In [11]:
# Check the max len 
ex_len = []
for i in train_df['text']:
  word_count = len(str(i).split(" "))
  ex_len.append(word_count)
print('max length of text: ', max(ex_len), 'words')

max length of text:  760 words


In [12]:
print(train_df.head())
print('--------------------------------')
print(test_df.head())

                                                    text                 label
29038  dit Caderousse, il nous a menti: il a dit qu'i...  [1.0, 0.0, 0.0, 0.0]
25786  Il ne put y parvenir: la grande porte, formée ...  [0.0, 0.0, 0.0, 1.0]
7732   Si quelqu'un lui avait dit [20]alors que cette...  [0.0, 0.0, 0.0, 1.0]
9797                                      --Pour le roi?  [1.0, 0.0, 0.0, 0.0]
20861  Quand la danse s'arrête, les tables se garniss...  [0.0, 0.0, 1.0, 0.0]
--------------------------------
                              text                 label
13460                     voici M.  [1.0, 0.0, 0.0, 0.0]
971                            Ah!  [0.0, 0.0, 0.0, 1.0]
12936  J'étais mouillé et fatigué.  [0.0, 0.0, 1.0, 0.0]
1747                  --Peut-être.  [1.0, 0.0, 0.0, 0.0]
11280              dit Caderousse.  [1.0, 0.0, 0.0, 0.0]


In [13]:
type(train_df)

pandas.core.frame.DataFrame

In [14]:
# Convertir les DataFrames en objets Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

### Tokenize
Tranfromer les blocks de texte en vecteur pour être donné à Bert

In [15]:
# Définir le modèle et charger le tokenizer
bert_model = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(bert_model)

/home/yop/project_dumas2/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# Appliquer la tokenisation aux datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 6695/6695 [00:00<00:00, 10479.55 examples/s]


### Préparer les étiquettes 

In [17]:
def convert_labels_to_indices(labels):
    return [np.argmax(label) for label in labels]

# Convertir les étiquettes one-hot en indices
train_dataset = train_dataset.map(lambda examples: {'labels': convert_labels_to_indices(examples['label'])}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': convert_labels_to_indices(examples['label'])}, batched=True)

# Réorganiser les colonnes pour être compatibles avec le modèle
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map: 100%|██████████| 6695/6695 [00:00<00:00, 58461.49 examples/s]


### Model 

In [27]:
# Réorganiser les colonnes pour être compatibles avec le modèle

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [32]:
# Charger le modèle BERT pré-entraîné et ajouter une couche de classification
num_labels = len(set(df['Label']))  # Assurez-vous que num_labels est correct
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

### Configuration de Bert

### Parameters

In [ ]:
# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # Répertoire pour sauvegarder les résultats
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir='./logs',            # Répertoire pour les logs
    logging_steps=10,                # Fréquence des logs
)

In [ ]:
# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# Entraîner le modèle
trainer.train()

In [ ]:
# Évaluer le modèle
results = trainer.evaluate()
print(results)